In [1]:
import math, re, os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from functools import partial
from PIL import Image

from sklearn.model_selection import train_test_split
print("Tensorflow version " + tf.__version__)

import tensorflow.keras.layers as L
import tensorflow.keras.backend as K

Tensorflow version 2.4.0


In [2]:
general_path = '../input/'

In [3]:
train = pd.read_csv(general_path + 'train.csv')
train['label'] = train['label'].apply(lambda x: str(x))#train['label'].astype('string')
train.sample(5)

,image_id,label
13423,3400845479.jpg,1
8588,2523649190.jpg,3
3330,1581037788.jpg,3
16549,3974644689.jpg,0
1927,1339403533.jpg,1


In [4]:
len(train)

21397

In [5]:
train['image_id'] = "../input/train_images/"+train['image_id']

In [7]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(feature0, feature1, feature2):
    feature = {
      'image': _bytes_feature(feature0),
      'image_name': _bytes_feature(feature1),
      'target': _int64_feature(feature2)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [8]:
from sklearn.model_selection import StratifiedKFold
import cv2

In [9]:
train['label'] = train['label'].astype(int)

In [10]:
train = train.reset_index(drop=True)

In [12]:
new_path = '../input/recreate_train_tfrecords/'

In [16]:
database_base_path = '/../input/'
PATH = f'{database_base_path}train_images/'
IMGS = train['image_id'].tolist()
N_FILES = 20
HEIGHT, WIDTH = (600, 800)
IMG_QUALITY = 100
print('No of Images - ', len(IMGS))
# train = df_merged.copy()
display(train.head())


folds = StratifiedKFold(n_splits=N_FILES, shuffle=True, random_state=123)
train['file'] = -1

for fold_n, (train_idx, val_idx) in enumerate(folds.split(train, train['label'])):
    print('File: %s has %s samples' % (fold_n+1, len(val_idx)))
    train['file'].loc[val_idx] = fold_n
    
train.to_csv(new_path + 'train.csv', index=False)

# Writing to TFRecords
for tfrec_num in range(N_FILES):
    print('\nWriting TFRecord %i of %i...'%(tfrec_num+1, N_FILES))
    samples = train[train['file'] == tfrec_num]
    n_samples = len(samples)
    print(f'{n_samples} samples')
    
    with tf.io.TFRecordWriter(new_path + 'Id_train%.2i-%i.tfrec'%(tfrec_num+1, n_samples)) as writer:
        for row in samples.itertuples():
            label = row.label
            image_name = row.image_id
            img_path = image_name
            
            img = cv2.imread(img_path)
            img = cv2.resize(img, (HEIGHT, WIDTH))
            img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, IMG_QUALITY))[1].tostring()
            
            example = serialize_example(img, str.encode(image_name), label)
            writer.write(example)

No of Images -  21397


,image_id,label,file
0,../input/train_images/1000015157.jpg,0,10
1,../input/train_images/1000201771.jpg,3,9
2,../input/train_images/100042118.jpg,1,0
3,../input/train_images/1000723321.jpg,1,8
4,../input/train_images/1000812911.jpg,3,19


File: 1 has 1070 samples
File: 2 has 1070 samples
File: 3 has 1070 samples
File: 4 has 1070 samples
File: 5 has 1070 samples
File: 6 has 1070 samples
File: 7 has 1070 samples
File: 8 has 1070 samples
File: 9 has 1070 samples
File: 10 has 1070 samples
File: 11 has 1070 samples
File: 12 has 1070 samples
File: 13 has 1070 samples
File: 14 has 1070 samples
File: 15 has 1070 samples
File: 16 has 1070 samples
File: 17 has 1070 samples
File: 18 has 1069 samples
File: 19 has 1069 samples
File: 20 has 1069 samples

Writing TFRecord 1 of 20...
1070 samples


/home/usmanr/.virtualenvs/tf-gpu/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/home/usmanr/.virtualenvs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.



Writing TFRecord 2 of 20...
1070 samples


...

/home/usmanr/.virtualenvs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.



Writing TFRecord 19 of 20...
1069 samples


/home/usmanr/.virtualenvs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.



Writing TFRecord 20 of 20...
1069 samples


/home/usmanr/.virtualenvs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
